<a href="https://colab.research.google.com/github/kagankazim/FreshAir/blob/master/FreshairPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas


İlk olarak geopandas librarysini ekliyoruz

In [0]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import pandas as pd
# gelen float veya string değerleri geo tipini dönüştürmek için geopandas kullanıyoruz
import geopandas as gpd
# Markerları ve harita arayüzünü oluşturmak için folium kütüphanesini kullanıyoruz
import folium
import math
from folium import Choropleth, Circle, Marker   
from folium.plugins import HeatMap, MarkerCluster
from folium import plugins
import matplotlib.pyplot as plt
import geopy.distance
import plotly.express as px



In [3]:
cred = credentials.Certificate('/content/drive/My Drive/ColabResource/freshair-491b9-firebase-adminsdk-9w0q4-76684634c1.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://freshair-491b9.firebaseio.com/'
})



In [0]:
ref = db.reference('/101/airQuality')



In [5]:
dataframe1= pd.DataFrame(ref.get()[:])
dataframe1.head()

,altitude,date,latitude,longitude,quality
0,0.0,140520,38.552118,27.04897,672
1,0.0,140520,38.552118,27.04897,668
2,0.0,140520,38.552118,27.04897,665
3,0.0,140520,38.552118,27.04897,656
4,0.0,140520,38.552118,27.04897,649


In [6]:
dataframe1.dtypes

altitude     float64
date           int64
latitude     float64
longitude    float64
quality        int64
dtype: object

In [0]:
dataframe1["date"].fillna(130520.0 , inplace=True)

In [0]:
m=folium.Map(width=500,height=500,location=[38.551946, 27.048199],tiles="openstreetmap",zoom_start=13)


In [0]:
for idx, row in dataframe1.iterrows():
    if(row.quality<400):
      color='green'
    if((row.quality>=400) & (row.quality<=700)):
      color='yellow'
    if(row.quality>700):
      color='red'
    folium.Circle(radius=row.quality/30, location=[row['latitude'], row['longitude']], popup= str(row.quality)+str(row.date), color=color, fill=True,).add_to(m)


In [10]:
m

In [0]:

dataframe1['DateTime'] = dataframe1['date'].apply(lambda x: str(x))
dataframe1['DateTime'] = dataframe1['DateTime'].apply(lambda x: x[0:2]+"-"+ x[2:4] +"-20"+ x[4:6])

In [12]:
dataframe1.head()

,altitude,date,latitude,longitude,quality,DateTime
0,0.0,140520,38.552118,27.04897,672,14-05-2020
1,0.0,140520,38.552118,27.04897,668,14-05-2020
2,0.0,140520,38.552118,27.04897,665,14-05-2020
3,0.0,140520,38.552118,27.04897,656,14-05-2020
4,0.0,140520,38.552118,27.04897,649,14-05-2020


In [0]:
#dateList =dataframe1["DateTime"].unique()
#datedf["date"]=pd.DataFrame(dateList)
#for each in datedf["date"]:
#  datedf["mean"]=dataframe1[[dataframe1["DateTime"]==each]].sum()

In [25]:
fig = px.density_mapbox(dataframe1, lat='latitude', lon='longitude', z='quality', radius=20,animation_frame="DateTime",title="Hava Kalitesi Haritası",
                        center=dict(lat=38.552118, lon=27.04897),width=800,height=600, zoom=13,
                        mapbox_style="open-street-map")
fig.show()